In [1]:
import polars as pl
import glob
import os
from pathlib import Path
import pandas as pd

root = Path.cwd().parent
root

PosixPath('/Users/skywings/Documents/RA/Professor_Zhao_RA/任务/chicago-taxi-data')

In [2]:
df_sample = pd.read_csv(root / 'input' / 'chicago_taxi_trips_000000000003.csv')
display(df_sample.head(), df_sample.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 23 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   unique_key              1000000 non-null  object 
 1   taxi_id                 1000000 non-null  object 
 2   trip_start_timestamp    1000000 non-null  object 
 3   trip_end_timestamp      1000000 non-null  object 
 4   trip_seconds            999628 non-null   float64
 5   trip_miles              1000000 non-null  float64
 6   pickup_census_tract     23825 non-null    float64
 7   dropoff_census_tract    35542 non-null    float64
 8   pickup_community_area   30401 non-null    float64
 9   dropoff_community_area  73067 non-null    float64
 10  fare                    999904 non-null   float64
 11  tips                    999904 non-null   float64
 12  tolls                   996680 non-null   float64
 13  extras                  999904 non-null   float64
 14  tri

,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,18b092309dff1fa8298b21f8dfb9be19124363c4,193f49f2e6f3c31a0058bb26e322f818043e6d64ff80a9...,2014-06-07 23:15:00 UTC,2014-06-07 23:15:00 UTC,0.0,0.0,1.703182e+10,1.703182e+10,NaN,NaN,...,NaN,NaN,Cash,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7b0601c2eaeccb6c7629f48858a007afea2e98e4,2cb7deb7674470467d31b1bba4657ab1c44c1feebf6274...,2014-04-16 15:30:00 UTC,2014-04-16 15:30:00 UTC,0.0,0.0,NaN,NaN,NaN,77.0,...,NaN,NaN,Credit Card,NaN,NaN,NaN,NaN,41.986712,-87.663416,POINT (-87.6634164054 41.9867117999)
2,0e1861cfb146b9e1b8b773b5be89f69cee92a0fc,e055c27835840cb1b08c8f68e20066307ab235a02fe7bd...,2014-04-25 14:15:00 UTC,2014-04-25 14:15:00 UTC,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,Credit Card,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,f8583c9d5d9fde21f8bcdd7b3ed0439c21a7fd88,e055c27835840cb1b08c8f68e20066307ab235a02fe7bd...,2014-04-25 14:15:00 UTC,2014-04-25 14:15:00 UTC,60.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,Credit Card,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5551bd37929928c32a1e703f871cedd1978709f0,e055c27835840cb1b08c8f68e20066307ab235a02fe7bd...,2014-04-25 16:30:00 UTC,2014-04-25 16:30:00 UTC,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,Credit Card,NaN,NaN,NaN,NaN,NaN,NaN,NaN


None

In [3]:

# -*- coding: utf-8 -*-

import os
import glob
import shutil  # 用于处理文件夹
import polars as pl
import pandas as pd
from tqdm import tqdm
from pathlib import Path

# 1. 标准结构定义 (与您原来的一致，无需改动)
# 定义所有列的统一数据类型和顺序
STANDARD_SCHEMA = {
    "unique_key": pl.Utf8, "taxi_id": pl.Utf8, "trip_start_timestamp": pl.Utf8,
    "trip_end_timestamp": pl.Utf8, "trip_seconds": pl.Float64, "trip_miles": pl.Float64,
    "pickup_census_tract": pl.Float64, "dropoff_census_tract": pl.Float64,
    "pickup_community_area": pl.Float64, "dropoff_community_area": pl.Float64,
    "fare": pl.Float64, "tips": pl.Float64, "tolls": pl.Float64,
    "extras": pl.Float64, "trip_total": pl.Float64, "payment_type": pl.Utf8,
    "company": pl.Utf8, "pickup_latitude": pl.Float64, "pickup_longitude": pl.Float64,
    "pickup_location": pl.Utf8, "dropoff_latitude": pl.Float64,
    "dropoff_longitude": pl.Float64, "dropoff_location": pl.Utf8
}
STANDARD_COLS = list(STANDARD_SCHEMA.keys())

# 2. 安全读取CSV的函数 (与您原来的一致，无需改动)
# 这个函数能健壮地处理列名不一致、列缺失等问题
def robust_csv_reader(file_path):
    """
    安全地读取单个CSV文件，处理可能的错误，并将其转换为标准化的Polars DataFrame。
    """
    try:
        # 使用 Polars 读取 CSV，初始全部作为字符串类型，避免推断错误
        df = pl.read_csv(
            file_path, infer_schema_length=0,
            dtypes={col: pl.Utf8 for col in STANDARD_COLS},
            null_values=["", "NA", "NULL", "N/A", "NaN", "."],
            ignore_errors=True
        )
        # 清理列名：转为小写，替换空格为下划线
        df = df.rename(lambda col: col.strip().lower().replace(" ", "_"))
        # 确保所有标准列都存在，不存在则用null填充
        for col in STANDARD_COLS:
            if col not in df.columns:
                df = df.with_columns(pl.lit(None, dtype=pl.Utf8).alias(col))
        # 选择标准列并按照标准schema进行类型转换
        return df.select(STANDARD_COLS).cast(STANDARD_SCHEMA, strict=False)
    except Exception as e:
        print(f"⚠️ 文件 {os.path.basename(file_path)} 读取失败: {str(e)}")
        return None

# 3. 主处理函数 (修改版：分块写入DTA，避免内存崩溃)
def merge_csv_to_stata_chunks(csv_dir, output_path_base, batch_size=20):
    """
    将CSV文件夹中的文件分批合并，并最终生成多个分块的 .dta 文件。
    这个方法可以处理非常大的数据集，因为它避免了一次性将所有数据加载到内存中。
    """
    # 创建一个临时目录来存放中间的 Parquet 文件
    temp_dir = Path(csv_dir) / "temp_parquet_chunks"
    if temp_dir.exists():
        shutil.rmtree(temp_dir) # 如果旧的临时目录存在，先清空
    temp_dir.mkdir()
    
    all_files = sorted(glob.glob(os.path.join(csv_dir, "*.csv")))
    print(f"📂 发现 {len(all_files)} 个CSV文件，将以每批 {batch_size} 个文件的规模处理。")

    if not all_files:
        print("🤷‍♀️ 在指定目录未找到任何CSV文件，操作终止。")
        return

    # --- 阶段一：将 CSV 分批转换为 Parquet 中间文件 ---
    total_batches = (len(all_files) + batch_size - 1) // batch_size
    for i in tqdm(range(total_batches), desc="处理CSV批次并生成Parquet"):
        start_idx = i * batch_size
        end_idx = start_idx + batch_size
        batch_files = all_files[start_idx:end_idx]
        
        batch_dfs = [robust_csv_reader(f) for f in batch_files if f]
        batch_dfs = [df for df in batch_dfs if df is not None and df.height > 0]

        if not batch_dfs:
            continue
            
        # 合并当前批次的 Polars DataFrames
        batch_polars_df = pl.concat(batch_dfs)
        
        # 将当前小批次立刻存为 Parquet 文件，内存占用低
        # Parquet 是一种高效的列式存储格式
        parquet_path = temp_dir / f"batch_{i:04d}.parquet"
        batch_polars_df.write_parquet(parquet_path)

    print("\n✅ 所有CSV批次均已处理并保存为临时的 Parquet 文件。")

    # --- 阶段二：逐一将 Parquet 中间文件转换为 .dta 分块文件 ---
    # 这是避免内存崩溃的关键步骤
    print("⏳ 开始将每个 Parquet 分块独立转换为 .dta 文件...")
    all_parquet_files = sorted(temp_dir.glob("*.parquet"))
    
    if not all_parquet_files:
        print("🤷‍♀️ 未能生成任何 Parquet 中间文件，无法继续。")
        shutil.rmtree(temp_dir)
        return

    try:
        for i, parquet_file in enumerate(tqdm(all_parquet_files, desc="转换Parquet分块为DTA")):
            # a. 只读取一个分块，内存占用小
            polars_df_chunk = pl.read_parquet(parquet_file)
            
            # b. 只将这个小分块转换为 Pandas，这是整个流程中内存占用最高点，但已被分块限制
            pandas_df_chunk = polars_df_chunk.to_pandas()
            
            # c. 将这个小分块写入一个独立的 .dta 文件
            output_dta_part = f"{output_path_base}_part_{i:04d}.dta"
            pandas_df_chunk.to_stata(output_dta_part, write_index=False)
        
        print(f"\n✅ 所有分块处理完毕！")
        print(f"📂 请在输出目录查看生成的 `..._part_N.dta` 文件。")

    except Exception as e:
        print(f"❌ 在转换 Parquet 到 DTA 的过程中发生错误: {e}")
        print("💡 请检查pandas和相关依赖是否正确安装。")
    
    finally:
        # 5. 清理临时文件
        print("🗑️ 正在清理临时文件...")
        shutil.rmtree(temp_dir)
        print("✨ 操作完成。")


# 4. 执行入口
if __name__ == "__main__":
    # --- 请在这里配置您的路径 ---
    # 使用 pathlib 来构建跨平台的路径
    # root = Path.cwd() # 如果脚本在项目根目录的子文件夹中，可能需要 Path.cwd().parent
    root = Path.cwd().parent 

    # 输入文件夹，存放您所有的.csv文件
    input_directory = str(root / 'input')
    
    # 输出文件夹和文件名前缀
    # 例如，这里会生成 merged_data_part_0000.dta, merged_data_part_0001.dta, ...
    output_file_base = str(root / 'output' / 'merged_data')

    # 检查并创建输出目录
    Path(root / 'output').mkdir(exist_ok=True)

    # --- 执行主函数 ---
    merge_csv_to_stata_chunks(
        csv_dir=input_directory,
        output_path_base=output_file_base,
        batch_size=10 #可以根据你的内存大小调整这个值，越小，单批次内存占用越低
    )

📂 发现 212 个CSV文件，将以每批 10 个文件的规模处理。


处理CSV批次并生成Parquet:   0%|          | 0/22 [00:00<?, ?it/s]/var/folders/gx/8b80zx6d2dd0slh0770_q04w0000gn/T/ipykernel_33918/1042761720.py:34: DeprecationWarning: the argument `dtypes` for `read_csv` is deprecated. It was renamed to `schema_overrides` in version 0.20.31.
  df = pl.read_csv(
处理CSV批次并生成Parquet: 100%|██████████| 22/22 [04:01<00:00, 10.96s/it]



✅ 所有CSV批次均已处理并保存为临时的 Parquet 文件。
⏳ 开始将每个 Parquet 分块独立转换为 .dta 文件...


转换Parquet分块为DTA: 100%|██████████| 22/22 [16:02<00:00, 43.77s/it]



✅ 所有分块处理完毕！
📂 请在输出目录查看生成的 `..._part_N.dta` 文件。
🗑️ 正在清理临时文件...
✨ 操作完成。
